In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box, Point
import numpy as np

In [62]:
df = pd.read_csv(
    "data/processed/bearing/ais_with_bearing_rate.csv",
    dtype={"MMSI": str},
    low_memory=False
)
print(f"📥  Ursprüngliche AIS-Meldungen geladen: {len(df):,}")

📥  Ursprüngliche AIS-Meldungen geladen: 977,539


In [66]:
df_raw = pd.read_csv(
    "data/processed/bearing/ais_with_bearing.csv",
    dtype={"MMSI": str},
    low_memory=False
)
print(f"📥  Ursprüngliche AIS-Meldungen geladen: {len(df_raw):,}")

📥  Ursprüngliche AIS-Meldungen geladen: 2,039,215


In [68]:
df_raw[df_raw["MMSI"]=="229344000"]

,# Timestamp,MMSI,Latitude,Longitude,Destination,bearing
0,2022-10-01 00:00:20,229344000,54.530143,11.356242,LTKLJ,141.098699
1,2022-10-01 00:00:26,229344000,54.529957,11.356872,LTKLJ,140.944377
4,2022-10-01 00:00:36,229344000,54.529730,11.357615,LTKLJ,140.769113
7,2022-10-01 00:00:47,229344000,54.529517,11.358303,LTKLJ,140.610630
9,2022-10-01 00:00:55,229344000,54.529357,11.358828,LTKLJ,140.489642
...,...,...,...,...,...,...
6275,2022-10-01 01:18:35,229344000,54.439713,11.668970,LTKLJ,121.984297
6294,2022-10-01 01:18:47,229344000,54.439510,11.669712,LTKLJ,121.971270
6321,2022-10-01 01:19:06,229344000,54.439130,11.671107,LTKLJ,121.946599
6336,2022-10-01 01:19:17,229344000,54.438933,11.671795,LTKLJ,121.936259


In [65]:
df.head(10)

,# Timestamp,MMSI,Latitude,Longitude,Destination,bearing,segment_idx
0,2022-10-01 00:00:20,229344000,54.530143,11.356242,LTKLJ,141.098699,0
1,2022-10-01 00:00:40,229344000,54.530143,11.356242,LTKLJ,141.098699,0
2,2022-10-01 00:01:00,229344000,54.530143,11.356242,LTKLJ,141.098699,0
3,2022-10-01 00:01:20,229344000,54.530143,11.356242,LTKLJ,141.098699,0
4,2022-10-01 00:01:40,229344000,54.530143,11.356242,LTKLJ,141.098699,0
5,2022-10-01 00:02:00,229344000,54.530143,11.356242,LTKLJ,141.098699,0
6,2022-10-01 00:02:20,229344000,54.530143,11.356242,LTKLJ,141.098699,0
7,2022-10-01 00:02:40,229344000,54.530143,11.356242,LTKLJ,141.098699,0
8,2022-10-01 00:03:00,229344000,54.530143,11.356242,LTKLJ,141.098699,0
9,2022-10-01 00:03:20,229344000,54.530143,11.356242,LTKLJ,141.098699,0


In [61]:
len(df_raw["MMSI"].unique())

1002

In [48]:
test = df_raw[df_raw["MMSI"]=="229344000"]

In [49]:
test

,# Timestamp,Type of mobile,MMSI,Latitude,Longitude,Navigational status,ROT,SOG,COG,Heading,...,Data source type,A,B,C,D,geometry,dist_to_port,bearing,angle_diff,bearing_rate
21,01/10/2022 00:00:06,Class A,229344000,54.530363,11.355553,Under way using engine,0.0,9.4,118.3,122.0,...,AIS,NaN,NaN,NaN,NaN,POINT (11.355553 54.530363),10781.611109,141.260007,0.000000e+00,0.000000e+00
61,01/10/2022 00:00:16,Class A,229344000,54.530143,11.356242,Under way using engine,0.0,9.4,118.1,122.0,...,AIS,NaN,NaN,NaN,NaN,POINT (11.356242 54.530143),10867.278659,141.098699,-1.613082e-01,-1.613082e-02
71,01/10/2022 00:00:20,Class A,229344000,54.530143,11.356242,Under way using engine,0.0,9.4,118.1,122.0,...,AIS,83.0,12.0,6.0,7.0,POINT (11.356242 54.530143),10867.278659,141.098699,4.319711e-16,4.319711e-17
91,01/10/2022 00:00:26,Class A,229344000,54.529957,11.356872,Under way using engine,0.0,9.4,117.8,122.0,...,AIS,83.0,12.0,6.0,7.0,POINT (11.356871999999997 54.52995700000001),10943.744708,140.944377,-1.543211e-01,-1.543211e-02
133,01/10/2022 00:00:36,Class A,229344000,54.529730,11.357615,Under way using engine,0.0,9.4,118.1,122.0,...,AIS,83.0,12.0,6.0,7.0,POINT (11.357614999999997 54.52973),11034.916774,140.769113,-1.752645e-01,-1.752645e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20429,01/10/2022 01:18:35,Class A,229344000,54.439713,11.668970,Under way using engine,0.0,9.3,115.4,117.0,...,AIS,83.0,12.0,6.0,7.0,POINT (11.66897 54.43971299999999),49494.393639,121.984297,-9.568231e-03,-9.568231e-04
20479,01/10/2022 01:18:47,Class A,229344000,54.439510,11.669712,Under way using engine,0.0,9.3,115.5,117.0,...,AIS,83.0,12.0,6.0,7.0,POINT (11.669712 54.439510000000006),49585.412932,121.971270,-1.302621e-02,-1.302621e-03
20571,01/10/2022 01:19:06,Class A,229344000,54.439130,11.671107,Under way using engine,0.0,9.4,114.9,118.0,...,AIS,83.0,12.0,6.0,7.0,POINT (11.671107 54.43912999999999),49756.380723,121.946599,-2.467098e-02,-2.467098e-03
20617,01/10/2022 01:19:17,Class A,229344000,54.438933,11.671795,Under way using engine,0.0,9.4,115.3,118.0,...,AIS,83.0,12.0,6.0,7.0,POINT (11.671794999999998 54.438933000000006),49841.605760,121.936259,-1.034052e-02,-1.034052e-03


In [52]:
df_raw["Ship type"].unique()

array(['Undefined', 'Tug', 'Cargo', 'Other', 'Passenger', 'SAR', 'Tanker',
       'Military', 'Pleasure', 'Sailing', 'Port tender', 'HSC',
       'Dredging', 'Spare 1', 'Law enforcement', 'Reserved',
       'Anti-pollution', 'Fishing', 'Towing', 'Towing long/wide', 'Pilot',
       'Not party to conflict'], dtype=object)

In [47]:
df_raw[df_raw["Ship type"]=="Cargo"]

,# Timestamp,Type of mobile,MMSI,Latitude,Longitude,Navigational status,ROT,SOG,COG,Heading,...,Data source type,A,B,C,D,geometry,dist_to_port,bearing,angle_diff,bearing_rate
71,01/10/2022 00:00:20,Class A,229344000,54.530143,11.356242,Under way using engine,0.0,9.4,118.1,122.0,...,AIS,83.0,12.0,6.0,7.0,POINT (11.356242 54.530143),10867.278659,141.098699,4.319711e-16,4.319711e-17
91,01/10/2022 00:00:26,Class A,229344000,54.529957,11.356872,Under way using engine,0.0,9.4,117.8,122.0,...,AIS,83.0,12.0,6.0,7.0,POINT (11.356871999999997 54.52995700000001),10943.744708,140.944377,-1.543211e-01,-1.543211e-02
117,01/10/2022 00:00:32,Class A,249225000,54.597683,11.221700,Under way using engine,-1.1,14.7,297.5,297.0,...,AIS,160.0,20.0,6.0,22.0,POINT (11.2217 54.597683),8943.831227,296.029010,-8.544545e-16,-8.544545e-17
119,01/10/2022 00:00:33,Class A,249225000,54.597840,11.221180,Under way using engine,0.0,14.6,297.4,297.0,...,AIS,160.0,20.0,6.0,22.0,POINT (11.221179999999999 54.59784),9007.280701,296.040656,1.164624e-02,1.164624e-03
133,01/10/2022 00:00:36,Class A,229344000,54.529730,11.357615,Under way using engine,0.0,9.4,118.1,122.0,...,AIS,83.0,12.0,6.0,7.0,POINT (11.357614999999997 54.52973),11034.916774,140.769113,-1.752645e-01,-1.752645e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9843838,31/10/2022 23:59:51,Class A,261000205,54.526382,11.434343,Under way using engine,0.0,9.5,296.9,297.0,...,AIS,81.0,9.0,7.0,7.0,POINT (11.434343 54.52638199999999),18648.596582,121.670007,2.263920e-02,2.263920e-03
9843840,31/10/2022 23:59:51,Class A,210393000,54.446743,11.668768,Under way using engine,0.0,8.9,111.9,115.0,...,AIS,52.0,12.0,5.0,5.0,POINT (11.668768 54.446743),48824.900496,120.674970,-1.309204e-02,-1.309204e-03
9843841,31/10/2022 23:59:52,Class A,249327000,54.470632,11.652800,Under way using engine,5.4,9.1,293.8,290.0,...,AIS,83.0,7.0,4.0,10.0,POINT (11.6528 54.470632),45209.980838,116.887293,7.358076e-03,7.358076e-04
9843845,31/10/2022 23:59:52,Class B,219007462,54.656530,11.348250,Unknown value,NaN,0.0,240.8,240.0,...,AIS,5.0,10.0,2.0,2.0,POINT (11.34825 54.65652999999999),18708.879893,23.782493,6.387569e-03,6.387569e-04


In [7]:
import geopandas as gpd
import shapely.geometry
from pyproj import Geod


In [22]:
# Beispiel: Erstellung von zwei Punkten (als Einzelpunkte)
point2 = shapely.geometry.Point(11.285, 54.556)  # Koordinaten (Lon, Lat)
point1 = shapely.geometry.Point(11.289, 54.578)

# Oder als GeoSeries
geometry = gpd.GeoSeries([point1, point2])

# Alternativ: GeoPandas Dataframe mit Koordinaten
data = {'geometry': [point1, point2]}
gdf = gpd.GeoDataFrame(data)


In [23]:
# Mit Pyproj (empfohlen für GeoPandas)
geod = Geod(ellps='WGS84')
azimuth_east, az21, distance = geod.inv(point1.x, point1.y, point2.x, point2.y)  # Peilung und Entfernung in Metern
print(f"Peilung: {azimuth_east}")
print(az21)
print(f"Entfernung: {distance}")



Peilung: -173.96755899055626
6.029181833308968
Entfernung: 2462.570760280555


In [ ]:
geod = Geod(ellps="WGS84")

# Angenommen gdf ist euer GeoDataFrame mit Point( lon, lat ) in EPSG:4326
def compute_bearing_pyproj(lon1, lat1, lon2, lat2):
    az12, az21, dist = geod.inv(lon1, lat1, lon2, lat2)
    # az12 ist die Vorwärts-Peilung von P1→P2 in Grad
    return az12

In [30]:
import numpy as np

def get_angle_diff(angle1_deg, angle2_deg):
    """
    Berechnet die gerichtete Winkeldifferenz im Bereich [-180, 180) Grad.
    
    :param angle1_deg: Erster Winkel in Grad
    :param angle2_deg: Zweiter Winkel in Grad
    :return: Winkeldifferenz in Grad
    """
    # Umwandlung in Bogenmaß und auf den Einheitskreis bringen (als komplexe Zahl)
    angle1_rad = np.deg2rad(angle1_deg)
    angle2_rad = np.deg2rad(angle2_deg)
    
    # Komplexe Division der beiden Winkel (entspricht Differenz der Winkel)
    diff_complex = np.exp(1j * angle1_rad) / np.exp(1j * angle2_rad)
    
    # Winkel (Argument) der komplexen Zahl zurück in Grad
    angle_diff_deg = np.rad2deg(np.angle(diff_complex))
    
    return angle_diff_deg

In [31]:
print(get_angle_diff(350, 10))  # Gibt -20.0 zurück
print(get_angle_diff(10, 350))  # Gibt 20.0 zurück

-20.000000000000004
20.000000000000004


In [34]:
# Zwei Winkeln in Grad
angle_prev = 350  # z.B. Schiff steht fast auf 360° bzw. Nord
angle_next = 10   # z.B. Schiff ist knapp östlich von Nord

# Naive Differenz
raw_diff = angle_next - angle_prev
print("Raw diff:", raw_diff, "°")
# → Raw diff: -340 °   (falsch!)


# Richtige Differenz in [-180, +180):
def wrap_to_180(diff):
    """Mappt jeden Winkel-Diff auf das Intervall (-180, +180]."""
    diff = (diff + 180) % 360 - 180
    return diff

wrapped_diff = wrap_to_180(raw_diff)
print("Wrapped diff:", wrapped_diff, "°")
# → Wrapped diff: 20 °   (korrekt: von 350° über 0° zu 10° sind +20°)

# Oder als Funktion, die direkt mit zwei Winkeln arbeitet:
def angle_diff(a1, a2):
    return wrap_to_180(a2 - a1)

print("Angle diff:", angle_diff(350, 10), "°")
# → Angle diff: 20 °

Raw diff: -340 °
Wrapped diff: 20 °
Angle diff: 20 °


In [88]:
import pandas as pd
df_cleand = pd.read_csv("data/processed/bearing/ais_cleaned.csv")

In [89]:
df_cleand[df_cleand["MMSI"]==229344000]

,# Timestamp,Type of mobile,MMSI,Latitude,Longitude,Navigational status,ROT,SOG,COG,Heading,...,Draught,Destination,ETA,Data source type,A,B,C,D,geometry,dist_to_port
0,2022-10-01 00:00:20,Class A,229344000,54.530143,11.356242,Under way using engine,0.0,9.4,118.1,122.0,...,5.9,LTKLJ,02/10/2022 09:00:00,AIS,83.0,12.0,6.0,7.0,POINT (11.356242 54.530143),10867.278659
1,2022-10-01 00:00:26,Class A,229344000,54.529957,11.356872,Under way using engine,0.0,9.4,117.8,122.0,...,5.9,LTKLJ,02/10/2022 09:00:00,AIS,83.0,12.0,6.0,7.0,POINT (11.356871999999997 54.52995700000001),10943.744708
4,2022-10-01 00:00:36,Class A,229344000,54.529730,11.357615,Under way using engine,0.0,9.4,118.1,122.0,...,5.9,LTKLJ,02/10/2022 09:00:00,AIS,83.0,12.0,6.0,7.0,POINT (11.357614999999997 54.52973),11034.916774
7,2022-10-01 00:00:47,Class A,229344000,54.529517,11.358303,Under way using engine,0.0,9.4,118.1,122.0,...,5.9,LTKLJ,02/10/2022 09:00:00,AIS,83.0,12.0,6.0,7.0,POINT (11.358303 54.52951699999999),11119.722109
9,2022-10-01 00:00:55,Class A,229344000,54.529357,11.358828,Under way using engine,0.0,9.4,117.8,122.0,...,5.9,LTKLJ,02/10/2022 09:00:00,AIS,83.0,12.0,6.0,7.0,POINT (11.358827999999999 54.529357),11184.141433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6275,2022-10-01 01:18:35,Class A,229344000,54.439713,11.668970,Under way using engine,0.0,9.3,115.4,117.0,...,5.9,LTKLJ,02/10/2022 09:00:00,AIS,83.0,12.0,6.0,7.0,POINT (11.66897 54.43971299999999),49494.393639
6294,2022-10-01 01:18:47,Class A,229344000,54.439510,11.669712,Under way using engine,0.0,9.3,115.5,117.0,...,5.9,LTKLJ,02/10/2022 09:00:00,AIS,83.0,12.0,6.0,7.0,POINT (11.669712 54.439510000000006),49585.412932
6321,2022-10-01 01:19:06,Class A,229344000,54.439130,11.671107,Under way using engine,0.0,9.4,114.9,118.0,...,5.9,LTKLJ,02/10/2022 09:00:00,AIS,83.0,12.0,6.0,7.0,POINT (11.671107 54.43912999999999),49756.380723
6336,2022-10-01 01:19:17,Class A,229344000,54.438933,11.671795,Under way using engine,0.0,9.4,115.3,118.0,...,5.9,LTKLJ,02/10/2022 09:00:00,AIS,83.0,12.0,6.0,7.0,POINT (11.671794999999998 54.438933000000006),49841.605760


In [97]:
df_inter = pd.read_csv("data/processed/bearing/ais_interpolated.csv")


In [98]:
df_inter.head(10)

,# Timestamp,Type of mobile,MMSI,Latitude,Longitude,Navigational status,ROT,SOG,COG,Heading,...,Destination,ETA,Data source type,A,B,C,D,geometry,dist_to_port,segment_idx
0,2022-10-01 00:00:20,Class A,229344000,54.530143,11.356242,Under way using engine,0.0,9.400000,118.100000,122.000000,...,LTKLJ,02/10/2022 09:00:00,AIS,83.0,12.0,6.0,7.0,POINT (11.356242 54.530143),10867.278659,0
1,2022-10-01 00:00:40,Class A,229344000,54.529653,11.357865,Under way using engine,0.0,9.400000,118.100000,122.000000,...,LTKLJ,02/10/2022 09:00:00,AIS,83.0,12.0,6.0,7.0,POINT (11.357614999999997 54.52973),11065.755078,0
2,2022-10-01 00:01:00,Class A,229344000,54.529249,11.359166,Under way using engine,0.0,9.400000,117.936364,122.454545,...,LTKLJ,02/10/2022 09:00:00,AIS,83.0,12.0,6.0,7.0,POINT (11.358827999999999 54.529357),11226.228924,0
3,2022-10-01 00:01:20,Class A,229344000,54.528824,11.360489,Under way using engine,0.0,9.400000,118.800000,123.000000,...,LTKLJ,02/10/2022 09:00:00,AIS,83.0,12.0,6.0,7.0,POINT (11.359572 54.52912),11391.401268,0
4,2022-10-01 00:01:40,Class A,229344000,54.528394,11.361816,Under way using engine,0.0,9.400000,119.270000,123.000000,...,LTKLJ,02/10/2022 09:00:00,AIS,83.0,12.0,6.0,7.0,POINT (11.361609999999999 54.52846),11557.524375,0
5,2022-10-01 00:02:00,Class A,229344000,54.527966,11.363148,Under way using engine,0.0,9.454545,119.090909,123.000000,...,LTKLJ,02/10/2022 09:00:00,AIS,83.0,12.0,6.0,7.0,POINT (11.362815 54.528077),11724.024668,0
6,2022-10-01 00:02:20,Class A,229344000,54.527549,11.364431,Under way using engine,0.0,9.400000,119.080000,123.000000,...,LTKLJ,02/10/2022 09:00:00,AIS,83.0,12.0,6.0,7.0,POINT (11.364179999999998 54.52762999999999),11885.017598,0
7,2022-10-01 00:02:40,Class A,229344000,54.527124,11.365741,Under way using engine,0.0,9.470000,118.870000,122.300000,...,LTKLJ,02/10/2022 09:00:00,AIS,83.0,12.0,6.0,7.0,POINT (11.365537 54.52718999999999),12049.394858,0
8,2022-10-01 00:03:00,Class A,229344000,54.526699,11.367084,Under way using engine,0.0,9.400000,118.800000,122.454545,...,LTKLJ,02/10/2022 09:00:00,AIS,83.0,12.0,6.0,7.0,POINT (11.366797999999998 54.52679299999999),12216.590315,0
9,2022-10-01 00:03:20,Class A,229344000,54.526289,11.368378,Under way using engine,0.0,9.400000,119.000000,123.000000,...,LTKLJ,02/10/2022 09:00:00,AIS,83.0,12.0,6.0,7.0,POINT (11.368107999999998 54.526377),12377.916949,0


In [100]:
df_bearing = pd.read_csv("data/processed/bearing/ais_with_bearing.csv")
df_bearing.head(10)

,# Timestamp,MMSI,Latitude,Longitude,Destination,bearing
0,2022-10-01 00:00:20,229344000,54.530143,11.356242,LTKLJ,141.098699
1,2022-10-01 00:00:40,229344000,54.529653,11.357865,LTKLJ,140.711241
2,2022-10-01 00:01:00,229344000,54.529249,11.359166,LTKLJ,140.414996
3,2022-10-01 00:01:20,229344000,54.528824,11.360489,LTKLJ,140.129870
4,2022-10-01 00:01:40,229344000,54.528394,11.361816,LTKLJ,139.852954
5,2022-10-01 00:02:00,229344000,54.527966,11.363148,LTKLJ,139.579425
6,2022-10-01 00:02:20,229344000,54.527549,11.364431,LTKLJ,139.325535
7,2022-10-01 00:02:40,229344000,54.527124,11.365741,LTKLJ,139.071425
8,2022-10-01 00:03:00,229344000,54.526699,11.367084,LTKLJ,138.811562
9,2022-10-01 00:03:20,229344000,54.526289,11.368378,LTKLJ,138.567674


In [104]:
df_bearing = pd.read_csv("data/processed/bearing/ais_with_distance.csv")
df_bearing

,# Timestamp,MMSI,Latitude,Longitude,Destination,bearing,bearing_rate
0,2022-10-01 00:00:20,229344000,54.530143,11.356242,LTKLJ,141.098699,0.000000
1,2022-10-01 00:00:40,229344000,54.529653,11.357865,LTKLJ,140.711241,-0.019373
2,2022-10-01 00:01:00,229344000,54.529249,11.359166,LTKLJ,140.414996,-0.014812
3,2022-10-01 00:01:20,229344000,54.528824,11.360489,LTKLJ,140.129870,-0.014256
4,2022-10-01 00:01:40,229344000,54.528394,11.361816,LTKLJ,139.852954,-0.013846
...,...,...,...,...,...,...,...
977534,2022-10-31 23:58:18,232031538,54.523757,11.425563,FRURO,124.593544,0.003647
977535,2022-10-31 23:58:38,232031538,54.524222,11.424030,FRURO,124.668957,0.003771
977536,2022-10-31 23:58:58,232031538,54.524740,11.422343,FRURO,124.750456,0.004075
977537,2022-10-31 23:59:18,232031538,54.525230,11.420743,FRURO,124.829905,0.003972
